In [2]:
import torch
from torch import nn
import torchvision.models as models
import torchvision
from revuresnet18 import revuresnet18

In [5]:
model = models.resnet18(pretrained = True)
print(model.fc)

Linear(in_features=512, out_features=1000, bias=True)


In [107]:
class Model3d():
    def __init__(self):
        
        self.model = models.resnet18(pretrained = True)
        
        self.model.conv1 = nn.Conv2d(4, 64, 7, stride=2, padding=3, bias=False)
        self.model.avgpool = nn.AdaptiveAvgPool2d(1)
        self.model.fc = nn.Linear(512, 200) #encode_dim =200 (latent vec dim)
        self.encoder = nn.Sequential(self.model)
        
        n_dims=200; 
        nf=512;
        
        self.decoder = nn.Sequential(

        nn.ConvTranspose3d(n_dims, nf, 4, stride=1, padding=0, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf, nf//2, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//2, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf//2, nf//4, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//4, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf//4, nf//8, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//8, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),

        nn.ConvTranspose3d(nf//8, nf//16, 4, stride=2, padding=1, dilation=1, groups=1, bias=True),
        nn.BatchNorm3d(nf//16, eps=1e-5, momentum=0.1, affine=True),
        nn.ReLU(inplace=True),
        nn.ConvTranspose3d(nf//16, 1, 4, stride=2, padding=1, dilation=1, groups=1, bias=True)
        )


    def forward(self,x):
        latent_vec = self.encoder(x.float())
        latent_vec = latent_vec.view(latent_vec.size(0), -1, 1, 1, 1)
        vox = self.decoder(latent_vec)
        
        return latent_vec, vox;
        
    
        

In [108]:
import numpy as np
import cv2

depth = cv2.imread("depth.png")
normal = cv2.imread("normal.png")
sil = cv2.imread("sil.png")



normal = normal.transpose(2,0,1).reshape(-1,3,480,480)
depth = depth.transpose(2,0,1)
depth = depth[0,:,:].reshape(-1,1,480,480)
sil = sil.transpose(2,0,1)
sil = sil[0,:,:].reshape(-1,1,480,480)

normal = torch.from_numpy(normal)
depth = torch.from_numpy(depth)
sil = torch.from_numpy(sil)


print("Normal:", normal.shape)
print("Depth:", depth.shape)
print("Silhouette:", sil.shape)


is_bg = sil <= 0 #self.silhou_thres
depth[is_bg] = 0
normal[is_bg.repeat(1, 3, 1, 1)] = 0 # NOTE: if old net2, set to white (100),
x = torch.cat((depth, normal), 1) # and swap depth and normal

model = Model3d()
L, V = model.forward(x)

Normal: torch.Size([1, 3, 480, 480])
Depth: torch.Size([1, 1, 480, 480])
Silhouette: torch.Size([1, 1, 480, 480])


In [109]:
print(L.shape)
print(V.shape)

torch.Size([1, 200, 1, 1, 1])
torch.Size([1, 1, 128, 128, 128])
